In [1]:
import logging
from rdflib import URIRef, Namespace
from humemai import MemorySystem

# Define the custom namespace for humemai ontology
humemai = Namespace("https://humem.ai/ontology/")

# Turn off debug logging
logging.getLogger("humemai").setLevel(logging.WARNING)

# Initialize the memory system
memory_system = MemorySystem()

# Define multiple triples
triples = [
    (
        URIRef("https://example.org/person/Alice"),
        URIRef("https://example.org/event/met"),
        URIRef("https://example.org/person/Bob"),
    ),  # 0
    (
        URIRef("https://example.org/person/Bob"),
        URIRef("https://example.org/event/met"),
        URIRef("https://example.org/person/Charlie"),
    ),  # 1
    (
        URIRef("https://example.org/person/Alice"),
        URIRef("https://example.org/relationship/knows"),
        URIRef("https://example.org/person/David"),
    ),  # 2
    (
        URIRef("https://example.org/person/Charlie"),
        URIRef("https://example.org/event/met"),
        URIRef("https://example.org/person/Eve"),
    ),  # 3
    (
        URIRef("https://example.org/person/David"),
        URIRef("https://example.org/event/invited"),
        URIRef("https://example.org/person/Alice"),
    ),  # 4
    (
        URIRef("https://example.org/person/Eve"),
        URIRef("https://example.org/relationship/worksWith"),
        URIRef("https://example.org/person/Bob"),
    ),  # 5
    (
        URIRef("https://example.org/person/Charlie"),
        URIRef("https://example.org/event/attended"),
        URIRef("https://example.org/person/Bob"),
    ),  # 6
    (
        URIRef("https://example.org/entity/Dog"),
        URIRef("https://example.org/relationship/is"),
        URIRef("https://example.org/entity/Animal"),
    ),  # 7
    (
        URIRef("https://example.org/person/Alice"),
        URIRef("https://example.org/relationship/owns"),
        URIRef("https://example.org/entity/Dog"),
    ),  # 8
    (
        URIRef("https://example.org/person/Eve"),
        URIRef("https://example.org/event/met"),
        URIRef("https://example.org/person/Alice"),
    ),  # 9
    (
        URIRef("https://example.org/person/John"),
        URIRef("https://example.org/relationship/loves"),
        URIRef("https://example.org/entity/Animal"),
    ),  # 10
]

# Define qualifiers for episodic memories
episodic_qualifiers_1 = {
    "location": "New York",
    "time": "2024-04-27T15:00:00",
    "emotion": "happy",
    "event": "Coffee meeting",
}
episodic_qualifiers_2 = {
    "location": "London",
    "time": "2024-05-01T10:00:00",
    "emotion": "excited",
    "event": "Conference meeting",
}
episodic_qualifiers_3 = {
    "location": "Paris",
    "time": "2024-05-03T14:00:00",
    "emotion": "curious",
    "event": "Workshop",
}

# Add long-term episodic memories
memory_system.memory.add_long_term_memory(
    "episodic", [triples[0]], **episodic_qualifiers_1
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[1]], **episodic_qualifiers_2
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[2]], **episodic_qualifiers_3
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[3]], **episodic_qualifiers_1
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[4]], **episodic_qualifiers_2
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[5]], **episodic_qualifiers_3
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[6]], **episodic_qualifiers_1
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[7]], **episodic_qualifiers_2
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[8]], **episodic_qualifiers_3
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[9]], **episodic_qualifiers_1
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[10]], **episodic_qualifiers_2
)


# Add a semantic memory for the "Dog" entity
semantic_qualifiers = {"derivedFrom": "animal_research", "strength": 5}
memory_system.memory.add_long_term_memory(
    "semantic", [triples[7]], **semantic_qualifiers
)
memory_system.memory.add_long_term_memory(
    "semantic", [triples[7]], **semantic_qualifiers
)
memory_system.memory.add_long_term_memory(
    "semantic", [triples[10]], **semantic_qualifiers
)
memory_system.memory.add_long_term_memory(
    "semantic", [triples[10]], **semantic_qualifiers
)

# Add a short-term memory (recent event)
memory_system.memory.add_short_term_memory([triples[8]], location="Alice's home")
memory_system.memory.add_short_term_memory([triples[8]], location="Alice's home")

# Add another short-term memory
memory_system.memory.add_short_term_memory([triples[9]], location="Paris Cafe")
memory_system.memory.add_short_term_memory([triples[9]], location="Paris Cafe")


# Function to count triples and memories in working memory
def print_memory_statistics(working_memory, hop_level):
    num_triples = working_memory.get_triple_count()
    num_memories = working_memory.get_memory_count()  # This counts reified statements

    print(f"\n===== Working Memory (Hop {hop_level}) =====")
    print(f"Number of Triples: {num_triples}")
    print(f"Number of Memories (Reified Statements): {num_memories}")
    print(repr(working_memory))


# Test working memory retrieval with different hops
trigger_node = URIRef("https://example.org/person/Alice")

# Hop 0 - Just the immediate node (only the short-term memories involving Alice)
working_memory_hop_0 = memory_system.get_working_memory(
    trigger_node=trigger_node, hops=0
)
print_memory_statistics(working_memory_hop_0, 0)

# Hop 1 - Immediate neighbors (both incoming and outgoing edges)
working_memory_hop_1 = memory_system.get_working_memory(
    trigger_node=trigger_node, hops=1
)
print_memory_statistics(working_memory_hop_1, 1)

# Hop 2 - 2-hop neighbors (further expansion of relationships)
working_memory_hop_2 = memory_system.get_working_memory(
    trigger_node=trigger_node, hops=2
)
print_memory_statistics(working_memory_hop_2, 2)

# Hop 3 - 3-hop neighbors (going even further in the memory graph)
working_memory_hop_3 = memory_system.get_working_memory(
    trigger_node=trigger_node, hops=3
)
print_memory_statistics(working_memory_hop_3, 3)

# Hop 4 - Maximum depth
working_memory_hop_4 = memory_system.get_working_memory(
    trigger_node=trigger_node, hops=4
)
print_memory_statistics(working_memory_hop_4, 4)

# Hop 5 - Maximum depth
working_memory_hop_5 = memory_system.get_working_memory(
    trigger_node=trigger_node, hops=5
)
print_memory_statistics(working_memory_hop_5, 5)

# Example: Get working memory with all long-term memories included
working_memory_all = memory_system.get_working_memory(include_all_long_term=True)
print_memory_statistics(working_memory_all, "All long-term")


===== Working Memory (Hop 0) =====
Number of Triples: 2
Number of Memories (Reified Statements): 4
[Eve, met, Alice, {'location': 'Paris Cafe', 'currentTime': '2024-10-03T17:06:11.690213', 'memoryID': '17'}]
[Eve, met, Alice, {'currentTime': '2024-10-03T17:06:11.690331', 'location': 'Paris Cafe', 'memoryID': '18'}]
[Alice, owns, Dog, {'location': "Alice's home", 'memoryID': '15', 'currentTime': '2024-10-03T17:06:11.690004'}]
[Alice, owns, Dog, {'currentTime': '2024-10-03T17:06:11.690118', 'location': "Alice's home", 'memoryID': '16'}]

===== Working Memory (Hop 1) =====
Number of Triples: 5
Number of Memories (Reified Statements): 9
[Alice, owns, Dog, {'memoryID': '15', 'location': "Alice's home", 'currentTime': '2024-10-03T17:06:11.690004'}]
[Alice, owns, Dog, {'currentTime': '2024-10-03T17:06:11.690118', 'location': "Alice's home", 'memoryID': '16'}]
[Eve, met, Alice, {'currentTime': '2024-10-03T17:06:11.690331', 'memoryID': '18', 'location': 'Paris Cafe'}]
[Eve, met, Alice, {'locat

In [2]:
memory_system

[Alice, met, Bob, {'memoryID': '0', 'location': 'New York', 'time': '2024-04-27T15:00:00', 'emotion': 'happy', 'event': 'Coffee meeting', 'recalled': '6'}]
[Bob, met, Charlie, {'memoryID': '1', 'location': 'London', 'time': '2024-05-01T10:00:00', 'emotion': 'excited', 'event': 'Conference meeting', 'recalled': '5'}]
[Alice, knows, David, {'memoryID': '2', 'location': 'Paris', 'time': '2024-05-03T14:00:00', 'emotion': 'curious', 'event': 'Workshop', 'recalled': '6'}]
[Charlie, met, Eve, {'memoryID': '3', 'location': 'New York', 'time': '2024-04-27T15:00:00', 'emotion': 'happy', 'event': 'Coffee meeting', 'recalled': '5'}]
[David, invited, Alice, {'memoryID': '4', 'location': 'London', 'time': '2024-05-01T10:00:00', 'emotion': 'excited', 'event': 'Conference meeting', 'recalled': '6'}]
[Eve, worksWith, Bob, {'memoryID': '5', 'location': 'Paris', 'time': '2024-05-03T14:00:00', 'emotion': 'curious', 'event': 'Workshop', 'recalled': '5'}]
[Charlie, attended, Bob, {'memoryID': '6', 'location

In [3]:
memory_system.memory.save_to_ttl("foo.ttl")

In [4]:
import logging
from rdflib import URIRef, Namespace
from humemai import MemorySystem

# Define the custom namespace for humemai ontology
humemai = Namespace("https://humem.ai/ontology/")

# Turn off debug logging
logging.getLogger("humemai").setLevel(logging.WARNING)

# Initialize the memory system
memory_system = MemorySystem()

memory_system.memory.load_from_ttl("foo.ttl")

memory_system

[Eve, met, Alice, {'currentTime': '2024-10-03T17:06:11.690331', 'location': 'Paris Cafe', 'memoryID': '18'}]
[Charlie, met, Eve, {'emotion': 'happy', 'event': 'Coffee meeting', 'location': 'New York', 'memoryID': '3', 'recalled': '5', 'time': '2024-04-27T15:00:00'}]
[David, invited, Alice, {'emotion': 'excited', 'event': 'Conference meeting', 'location': 'London', 'memoryID': '4', 'recalled': '6', 'time': '2024-05-01T10:00:00'}]
[Alice, met, Bob, {'emotion': 'happy', 'event': 'Coffee meeting', 'location': 'New York', 'memoryID': '0', 'recalled': '6', 'time': '2024-04-27T15:00:00'}]
[John, loves, Animal, {'emotion': 'excited', 'event': 'Conference meeting', 'location': 'London', 'memoryID': '10', 'recalled': '4', 'time': '2024-05-01T10:00:00'}]
[John, loves, Animal, {'derivedFrom': 'animal_research', 'memoryID': '14', 'recalled': '4', 'strength': '5'}]
[Dog, is, Animal, {'derivedFrom': 'animal_research', 'memoryID': '12', 'recalled': '5', 'strength': '5'}]
[Eve, met, Alice, {'currentTim

In [5]:
memory_system.memory

[Eve, met, Alice, {'currentTime': '2024-10-03T17:06:11.690331', 'location': 'Paris Cafe', 'memoryID': '18'}]
[Charlie, met, Eve, {'emotion': 'happy', 'event': 'Coffee meeting', 'location': 'New York', 'memoryID': '3', 'recalled': '5', 'time': '2024-04-27T15:00:00'}]
[David, invited, Alice, {'emotion': 'excited', 'event': 'Conference meeting', 'location': 'London', 'memoryID': '4', 'recalled': '6', 'time': '2024-05-01T10:00:00'}]
[Alice, met, Bob, {'emotion': 'happy', 'event': 'Coffee meeting', 'location': 'New York', 'memoryID': '0', 'recalled': '6', 'time': '2024-04-27T15:00:00'}]
[John, loves, Animal, {'emotion': 'excited', 'event': 'Conference meeting', 'location': 'London', 'memoryID': '10', 'recalled': '4', 'time': '2024-05-01T10:00:00'}]
[John, loves, Animal, {'derivedFrom': 'animal_research', 'memoryID': '14', 'recalled': '4', 'strength': '5'}]
[Dog, is, Animal, {'derivedFrom': 'animal_research', 'memoryID': '12', 'recalled': '5', 'strength': '5'}]
[Eve, met, Alice, {'currentTim

In [6]:
memory_system.memory.get_triple_count()

11

In [7]:
memory_system.memory.get_memory_count()

19

In [8]:
memory_system.memory.get_short_term_memory_count()

4

In [9]:
memory_system.memory.get_long_term_memory_count()

15

In [10]:
memory_system.memory.get_long_term_episodic_memory_count()

11

In [11]:
memory_system.memory.get_long_term_semantic_memory_count()

4

In [12]:
for memory in memory_system.memory.iterate_memories("short_term"):
    print(memory)

(rdflib.term.URIRef('https://example.org/person/Eve'), rdflib.term.URIRef('https://example.org/event/met'), rdflib.term.URIRef('https://example.org/person/Alice'), {'https://humem.ai/ontology/currentTime': '2024-10-03T17:06:11.690331', 'https://humem.ai/ontology/location': 'Paris Cafe', 'https://humem.ai/ontology/memoryID': '18'})
(rdflib.term.URIRef('https://example.org/person/Eve'), rdflib.term.URIRef('https://example.org/event/met'), rdflib.term.URIRef('https://example.org/person/Alice'), {'https://humem.ai/ontology/currentTime': '2024-10-03T17:06:11.690213', 'https://humem.ai/ontology/location': 'Paris Cafe', 'https://humem.ai/ontology/memoryID': '17'})
(rdflib.term.URIRef('https://example.org/person/Alice'), rdflib.term.URIRef('https://example.org/relationship/owns'), rdflib.term.URIRef('https://example.org/entity/Dog'), {'https://humem.ai/ontology/currentTime': '2024-10-03T17:06:11.690118', 'https://humem.ai/ontology/location': "Alice's home", 'https://humem.ai/ontology/memoryID'

In [13]:
len(list(memory_system.memory.iterate_memories("long_term")))

15

In [14]:
for memory in memory_system.memory.iterate_memories("long_term"):
    print(memory)

(rdflib.term.URIRef('https://example.org/person/Charlie'), rdflib.term.URIRef('https://example.org/event/met'), rdflib.term.URIRef('https://example.org/person/Eve'), {'https://humem.ai/ontology/emotion': 'happy', 'https://humem.ai/ontology/event': 'Coffee meeting', 'https://humem.ai/ontology/location': 'New York', 'https://humem.ai/ontology/memoryID': '3', 'https://humem.ai/ontology/recalled': '5', 'https://humem.ai/ontology/time': '2024-04-27T15:00:00'})
(rdflib.term.URIRef('https://example.org/person/David'), rdflib.term.URIRef('https://example.org/event/invited'), rdflib.term.URIRef('https://example.org/person/Alice'), {'https://humem.ai/ontology/emotion': 'excited', 'https://humem.ai/ontology/event': 'Conference meeting', 'https://humem.ai/ontology/location': 'London', 'https://humem.ai/ontology/memoryID': '4', 'https://humem.ai/ontology/recalled': '6', 'https://humem.ai/ontology/time': '2024-05-01T10:00:00'})
(rdflib.term.URIRef('https://example.org/person/Alice'), rdflib.term.URI

In [15]:
for memory in memory_system.memory.iterate_memories("episodic"):
    print(memory)

(rdflib.term.URIRef('https://example.org/person/Charlie'), rdflib.term.URIRef('https://example.org/event/met'), rdflib.term.URIRef('https://example.org/person/Eve'), {'https://humem.ai/ontology/emotion': 'happy', 'https://humem.ai/ontology/event': 'Coffee meeting', 'https://humem.ai/ontology/location': 'New York', 'https://humem.ai/ontology/memoryID': '3', 'https://humem.ai/ontology/recalled': '5', 'https://humem.ai/ontology/time': '2024-04-27T15:00:00'})
(rdflib.term.URIRef('https://example.org/person/David'), rdflib.term.URIRef('https://example.org/event/invited'), rdflib.term.URIRef('https://example.org/person/Alice'), {'https://humem.ai/ontology/emotion': 'excited', 'https://humem.ai/ontology/event': 'Conference meeting', 'https://humem.ai/ontology/location': 'London', 'https://humem.ai/ontology/memoryID': '4', 'https://humem.ai/ontology/recalled': '6', 'https://humem.ai/ontology/time': '2024-05-01T10:00:00'})
(rdflib.term.URIRef('https://example.org/person/Alice'), rdflib.term.URI

In [16]:
for memory in memory_system.memory.iterate_memories("semantic"):
    print(memory)

(rdflib.term.URIRef('https://example.org/person/John'), rdflib.term.URIRef('https://example.org/relationship/loves'), rdflib.term.URIRef('https://example.org/entity/Animal'), {'https://humem.ai/ontology/derivedFrom': 'animal_research', 'https://humem.ai/ontology/memoryID': '14', 'https://humem.ai/ontology/recalled': '4', 'https://humem.ai/ontology/strength': '5'})
(rdflib.term.URIRef('https://example.org/entity/Dog'), rdflib.term.URIRef('https://example.org/relationship/is'), rdflib.term.URIRef('https://example.org/entity/Animal'), {'https://humem.ai/ontology/derivedFrom': 'animal_research', 'https://humem.ai/ontology/memoryID': '12', 'https://humem.ai/ontology/recalled': '5', 'https://humem.ai/ontology/strength': '5'})
(rdflib.term.URIRef('https://example.org/entity/Dog'), rdflib.term.URIRef('https://example.org/relationship/is'), rdflib.term.URIRef('https://example.org/entity/Animal'), {'https://humem.ai/ontology/derivedFrom': 'animal_research', 'https://humem.ai/ontology/memoryID': 

In [17]:
len(list(memory_system.memory.iterate_memories()))

19

In [18]:
for memory in memory_system.memory.iterate_memories():
    print(memory)

(rdflib.term.URIRef('https://example.org/person/Eve'), rdflib.term.URIRef('https://example.org/event/met'), rdflib.term.URIRef('https://example.org/person/Alice'), {'https://humem.ai/ontology/currentTime': '2024-10-03T17:06:11.690331', 'https://humem.ai/ontology/location': 'Paris Cafe', 'https://humem.ai/ontology/memoryID': '18'})
(rdflib.term.URIRef('https://example.org/person/Charlie'), rdflib.term.URIRef('https://example.org/event/met'), rdflib.term.URIRef('https://example.org/person/Eve'), {'https://humem.ai/ontology/emotion': 'happy', 'https://humem.ai/ontology/event': 'Coffee meeting', 'https://humem.ai/ontology/location': 'New York', 'https://humem.ai/ontology/memoryID': '3', 'https://humem.ai/ontology/recalled': '5', 'https://humem.ai/ontology/time': '2024-04-27T15:00:00'})
(rdflib.term.URIRef('https://example.org/person/David'), rdflib.term.URIRef('https://example.org/event/invited'), rdflib.term.URIRef('https://example.org/person/Alice'), {'https://humem.ai/ontology/emotion':

In [19]:
memory

(rdflib.term.URIRef('https://example.org/person/Alice'),
 rdflib.term.URIRef('https://example.org/relationship/knows'),
 rdflib.term.URIRef('https://example.org/person/David'),
 {'https://humem.ai/ontology/emotion': 'curious',
  'https://humem.ai/ontology/event': 'Workshop',
  'https://humem.ai/ontology/location': 'Paris',
  'https://humem.ai/ontology/memoryID': '2',
  'https://humem.ai/ontology/recalled': '6',
  'https://humem.ai/ontology/time': '2024-05-03T14:00:00'})